In [1]:
import pandas as pd
import requests
import io
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)


def reader(url):
  url = url
  s = requests.get(url).content
  t = io.StringIO(s.decode('utf-8'))
  ds = pd.read_csv(t)
  return(ds)

data_2019 = reader("https://www-static.bouldercolorado.gov/docs/opendata/CouncilEmails_PlainText2019.csv")
data_2020 = reader("https://www-static.bouldercolorado.gov/docs/opendata/CouncilEmails_PlainText2020.csv")

Note that this is the second iteration created. I did the first iteration of the LDA model in R however, I was coming up against memory constraints using my personal computer. I switched over to a python environment so I could use the free Google Colab tool and have access to cloud compute.

## Data Cleaning

The data cleaning we do is pretty simple.

1. We remove all no reply type emails. These are typically automated emails that aren't relevant to this study.
1. We remove all stop words that do not provide value in a probablistic model such as LDA
1. We stem and lemmatize the documents.


In [11]:
display(data_2019)

,SentFrom,SentTo,SentCC,ReceivedDate,EmailSubject,PlainTextBody,MessageIdentifier
0,Petition For Boulder Homeless Services,Council,NaN,2019-12-31 22:44:46.0000000 +00:00,"Petition For Homeless Services, Signature Numb...","Dear Boulder City Council members, We are writ...",AAMkADQ2ZmVlYWI4LWI1MmEtNDc1NC05ZjhkLTI5YTA3ZD...
1,Petition For Boulder Homeless Services,Council,NaN,2019-12-31 20:20:04.0000000 +00:00,"Petition For Homeless Services, Signature Numb...","Dear Boulder City Council members, We are writ...",AAMkADQ2ZmVlYWI4LWI1MmEtNDc1NC05ZjhkLTI5YTA3ZD...
2,Petition For Boulder Homeless Services,Council,NaN,2019-12-31 19:28:02.0000000 +00:00,"Petition For Homeless Services, Signature Numb...","Dear Boulder City Council members, We are writ...",AAMkADQ2ZmVlYWI4LWI1MmEtNDc1NC05ZjhkLTI5YTA3ZD...
3,Petition For Boulder Homeless Services,Council,NaN,2019-12-31 15:37:14.0000000 +00:00,"Petition For Homeless Services, Signature Numb...","Dear Boulder City Council members, We are writ...",AAMkADQ2ZmVlYWI4LWI1MmEtNDc1NC05ZjhkLTI5YTA3ZD...
4,Petition For Boulder Homeless Services,Council,NaN,2019-12-31 15:26:31.0000000 +00:00,"Petition For Homeless Services, Signature Numb...","Dear Boulder City Council members, We are writ...",AAMkADQ2ZmVlYWI4LWI1MmEtNDc1NC05ZjhkLTI5YTA3ZD...
...,...,...,...,...,...,...,...
7630,David Figueroa,"Ana Vangelena, Council",NaN,2019-01-01 13:36:34.0000000 +00:00,Jon Benet Ramsey 2019,“My vision concerning Jon Benet Ramsey By Davi...,AAMkADQ2ZmVlYWI4LWI1MmEtNDc1NC05ZjhkLTI5YTA3ZD...
7631,Meagan Arango,Council,NaN,2019-01-01 11:51:35.0000000 +00:00,Severe Weather Shelter - Support ASAP,"Greetings Council, and Happy New Year to you. ...",AAMkADQ2ZmVlYWI4LWI1MmEtNDc1NC05ZjhkLTI5YTA3ZD...
7632,Max Weller,Council,NaN,2019-01-01 11:45:27.0000000 +00:00,STAY SOBER and stay alive outdoors!,"Dear Council members, Same message I've been p...",AAMkADQ2ZmVlYWI4LWI1MmEtNDc1NC05ZjhkLTI5YTA3ZD...
7633,Kenneth Flowe,Council,NaN,2019-01-01 08:54:04.0000000 +00:00,Updated: 2019 Martin Luther King Day Talent Sh...,This is a note to invite you to a very excitin...,AAMkADQ2ZmVlYWI4LWI1MmEtNDc1NC05ZjhkLTI5YTA3ZD...


In [7]:

from nltk.corpus import stopwords
stop = stopwords.words('english')
stemmer = SnowballStemmer('english')
def cleaner(raw_data):
    data = raw_data[raw_data.SentFrom != "No Reply"]
    data = data.dropna(subset=['PlainTextBody'])
    return(data)

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result    
    
    
#.apply(lambda x: [item for item in x if item not in stop])   
#data = cleaner( 
#    pd.concat([data_2019, data_2020]))

data = cleaner(pd.concat([data_2019, data_2020]))
#display(data)
processed_docs = data['PlainTextBody'].map(preprocess)



In [12]:
processed_docs[:10]

0    [dear, boulder, citi, council, member, write, ...
1    [dear, boulder, citi, council, member, write, ...
2    [dear, boulder, citi, council, member, write, ...
3    [dear, boulder, citi, council, member, write, ...
4    [dear, boulder, citi, council, member, write, ...
5    [dear, boulder, citi, council, member, write, ...
6    [dear, boulder, citi, council, member, write, ...
7    [dear, boulder, citi, council, member, write, ...
8    [dear, boulder, citi, council, member, write, ...
9    [hello, council, member, write, today, respons...
Name: PlainTextBody, dtype: object

## Generate Bag of Words


In [14]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
        
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


0 absorb
1 access
2 addit
3 address
4 adjud
5 affili
6 appoint
7 appropri
8 area
9 aris
10 base


In [18]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]


bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 38 ("fund") appears 4 time.
Word 39 ("help") appears 2 time.
Word 48 ("million") appears 1 time.
Word 54 ("open") appears 4 time.
Word 69 ("repres") appears 1 time.
Word 91 ("write") appears 1 time.
Word 92 ("year") appears 1 time.
Word 132 ("general") appears 1 time.
Word 137 ("hope") appears 1 time.
Word 142 ("increas") appears 1 time.
Word 147 ("major") appears 1 time.
Word 153 ("opportun") appears 1 time.
Word 176 ("today") appears 1 time.
Word 184 ("sarah") appears 2 time.
Word 222 ("kind") appears 1 time.
Word 229 ("recent") appears 2 time.
Word 286 ("consid") appears 1 time.
Word 356 ("includ") appears 1 time.
Word 394 ("park") appears 5 time.
Word 414 ("quick") appears 1 time.
Word 422 ("requir") appears 1 time.
Word 452 ("town") appears 1 time.
Word 457 ("uniqu") appears 1 time.
Word 460 ("user") appears 1 time.
Word 468 ("work") appears 1 time.
Word 497 ("rais") appears 3 time.
Word 498 ("rang") appears 1 time.
Word 529 ("annual") appears 1 time.
Word 596 ("councilwoman"

## Running LDA using Bag of Words
Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [23]:
num_topics = 15

lda_model = gensim.models.LdaMulticore(
    bow_corpus, 
    num_topics=num_topics, 
    id2word=dictionary, 
    passes=2, 
    workers=3)

KeyboardInterrupt: 

In [21]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.013*"open" + 0.012*"space" + 0.012*"flood" + 0.009*"plan" + 0.009*"protect" + 0.009*"year" + 0.008*"south" + 0.008*"communiti" + 0.007*"muni" + 0.007*"cost"
Topic: 1 
Words: 0.007*"plan" + 0.006*"public" + 0.006*"communiti" + 0.006*"build" + 0.006*"meet" + 0.006*"area" + 0.006*"need" + 0.005*"work" + 0.005*"project" + 0.005*"time"
Topic: 2 
Words: 0.046*"polic" + 0.024*"communiti" + 0.022*"depart" + 0.013*"hous" + 0.011*"resid" + 0.010*"support" + 0.010*"budget" + 0.010*"offic" + 0.009*"fund" + 0.007*"black"
Topic: 3 
Words: 0.043*"polic" + 0.039*"hous" + 0.035*"depart" + 0.030*"resid" + 0.027*"budget" + 0.027*"fund" + 0.025*"communiti" + 0.020*"afford" + 0.018*"harass" + 0.017*"limit"
Topic: 4 
Words: 0.009*"prairi" + 0.008*"email" + 0.006*"manag" + 0.006*"public" + 0.006*"land" + 0.006*"send" + 0.006*"work" + 0.006*"dog" + 0.005*"time" + 0.005*"open"
Topic: 5 
Words: 0.010*"polic" + 0.009*"census" + 0.007*"state" + 0.007*"offic" + 0.007*"https" + 0.005*"bike" + 0.0

In [ ]:
top_topics = lda_model.top_topics(bow_corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)